In [315]:
class BinaryTree:
    def __init__(self, key, leftTree=None, rightTree=None):
        self.key = key
        self.leftTree = leftTree
        self.rightTree = rightTree

    def setKey(self, key):
        self.key = key

    def getKey(self):
        return self.key

    def getLeftTree(self):
        return self.leftTree

    def getRightTree(self):
        return self.rightTree

    def insertLeft(self, key):
        if self.leftTree is None:
            self.leftTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, leftTree=self.leftTree)
            self.leftTree = new_node

    def insertRight(self, key):
        if self.rightTree is None:
            self.rightTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, rightTree=self.rightTree)
            self.rightTree = new_node

    def printPreorder(self, level=0):
        print(str(level * '-') + str(self.key))
        if self.leftTree is not None:
            self.leftTree.printPreorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPreorder(level + 1)
            
    def printInorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printInorder(level + 1)
        print(str(level * '-') + str(self.key))
        if self.rightTree is not None:
            self.rightTree.printInorder(level + 1)

    def printPostorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printPostorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPostorder(level + 1)
        print(str(level * '-') + str(self.key))


class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.isEmpty():
            return self.items.pop()
        else:
            raise IndexError("Pop from an empty stack")

    def peek(self):
        if not self.isEmpty():
            return self.items[-1]
        else:
            raise IndexError("Peek from an empty stack")

    def size(self):
        return len(self.items)


class Node:
    def __init__(self, key, item):
        self.key = key
        self.item = item
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, item):
        new_node = Node(key, item)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    def find(self, key):
        current = self.head
        while current:
            if current.key == key:
                return current.item
            current = current.next
        return None

    def sort(self):
        if self.head is None or self.head.next is None:
            return

        size = 1
        current = self.head
        while current.next:
            size += 1
            current = current.next

        step = 1
        while step < size:
            self.head = self.merge_pass(self.head, step)
            step *= 2

    def merge_pass(self, head, step):
        current = head
        prev_tail = None
        new_head = None

        while current:
            left = current
            right = self.split(left, step)
            current = self.split(right, step)

            merged = self.sorted_merge(left, right)

            if prev_tail:
                prev_tail.next = merged
            else:
                new_head = merged

            while prev_tail and prev_tail.next:
                prev_tail = prev_tail.next

        return new_head

    def split(self, head, step):
        if head is None:
            return None
        for _ in range(step - 1):
            if head.next is None:
                break
            head = head.next
        next_head = head.next
        head.next = None
        return next_head

    def sorted_merge(self, a, b):
        if a is None:
            return b
        if b is None:
            return a

        if a.key <= b.key:
            result = a
            result.next = self.sorted_merge(a.next, b)
        else:
            result = b
            result.next = self.sorted_merge(a, b.next)
        return result

In [316]:
class ParseTree:
    def __init__(self):
        self.operators = set(['+', '-', '*', '/', '**'])
        self.precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '**': 3}

    def tokenise(self, expression):
        exp = expression.replace(" ", "")
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif exp[i] in self.operators:
                if exp[i] == '-' and (i == 0 or exp[i - 1] in self.operators or exp[i - 1] == '('):
                    tokens.append('(')  # Start parenthesis for negative number
                    tokens.append('-1')
                    tokens.append('*')
                    j = i + 1
                    while j < len(exp) and (exp[j].isalnum() or exp[j] == '.'):
                        j += 1
                    if j < len(exp) and exp[j] == '(':
                        # Find the closing parenthesis for a negative function call
                        count = 1
                        j += 1
                        while j < len(exp) and count != 0:
                            if exp[j] == '(':
                                count += 1
                            elif exp[j] == ')':
                                count -= 1
                            j += 1
                    tokens.append(exp[i+1:j])  # Add the number or function call
                    tokens.append(')')  # End parenthesis for negative number
                    i = j - 1
                else:
                    tokens.append(exp[i])
                i += 1
            elif exp[i].isalnum() or exp[i] == '.':
                j = i
                while j < len(exp) and (exp[j].isalnum() or exp[j] == '.'):
                    j += 1
                tokens.append(exp[i:j])
                i = j
            else:
                i += 1
        print(tokens)
        return tokens

    def infix_to_postfix(self, exp):
        tokens = self.tokenise(exp)
        stack = Stack()
        postfix = []
        for token in tokens:
            if token.isalnum() or (token.startswith('-') and token[1:].isdigit()):
                postfix.append(token)
            elif token == '(':
                stack.push(token)
            elif token == ')':
                while not stack.isEmpty() and stack.peek() != '(':
                    postfix.append(stack.pop())
                stack.pop()  # Pop '(' from stack
            else:
                while (not stack.isEmpty()) and self.precedence.get(token, 0) <= self.precedence.get(stack.peek(), 0):
                    postfix.append(stack.pop())
                stack.push(token)

        while not stack.isEmpty():
            postfix.append(stack.pop())

        return postfix

    def evaluate_postfix(self, postfix, variables):
        stack = Stack()
        for token in postfix:
            if token.lstrip('-').replace('.', '').isdigit():
                # Direct number (including negative numbers)
                stack.push(token)
            elif token.isalpha():
                # Variable
                value = variables.find(token)
                if value is None:
                    return None  # Variable not found
                stack.push(value)
            else:
                # Perform operation
                b = stack.pop()
                a = stack.pop() if not stack.isEmpty() else '0'
                if token == '+':
                    stack.push(str(float(a) + float(b)))
                elif token == '-':
                    stack.push(str(float(a) - float(b)))
                elif token == '*':
                    stack.push(str(float(a) * float(b)))
                elif token == '/':
                    stack.push(str(float(a) / float(b)))
                elif token == '**':
                    stack.push(str(float(a) ** float(b)))

        return stack.pop()

In [318]:
# Test with LinkedList of expressions
expression_list = LinkedList()
expression_list.append('a', '1 * 2')
expression_list.append('b', 'a * 1')
expression_list.append('c', '-(5 * (-5))')
expression_list.append('d', 'x')

parse_tree = ParseTree()

for key in ['a', 'b', 'c', 'd']:
    expression = expression_list.find(key)
    if expression:
        postfix = parse_tree.infix_to_postfix(expression)
        result = parse_tree.evaluate_postfix(postfix, expression_list)
        print(f'{key} = {expression} => {result}')
    else:
        print(f'{key} = None')

['1', '*', '2']
a = 1 * 2 => 2.0
['a', '*', '1']


ValueError: could not convert string to float: '1 * 2'